In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

/Users/jasmine/Documents/PythonProject/learn/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 50000/50000 [00:00<00:00, 958996.17 examples/s]


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:09<00:00, 5186.96 examples/s]


In [3]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

                                                 
 33%|███▎      | 125/375 [03:15<05:02,  1.21s/it]

{'eval_loss': 1.1533335447311401, 'eval_accuracy': 0.468, 'eval_runtime': 45.649, 'eval_samples_per_second': 21.906, 'eval_steps_per_second': 2.738, 'epoch': 1.0}


                                                   
 67%|██████▋   | 250/375 [06:20<02:21,  1.13s/it]

{'eval_loss': 1.0128626823425293, 'eval_accuracy': 0.571, 'eval_runtime': 45.7782, 'eval_samples_per_second': 21.844, 'eval_steps_per_second': 2.731, 'epoch': 2.0}


                                                 
100%|██████████| 375/375 [09:26<00:00,  1.51s/it]

{'eval_loss': 1.0707471370697021, 'eval_accuracy': 0.59, 'eval_runtime': 41.5094, 'eval_samples_per_second': 24.091, 'eval_steps_per_second': 3.011, 'epoch': 3.0}
{'train_runtime': 566.8262, 'train_samples_per_second': 5.293, 'train_steps_per_second': 0.662, 'train_loss': 0.9688461100260417, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=0.9688461100260417, metrics={'train_runtime': 566.8262, 'train_samples_per_second': 5.293, 'train_steps_per_second': 0.662, 'total_flos': 789354427392000.0, 'train_loss': 0.9688461100260417, 'epoch': 3.0})